先导入URLs列表，看下大体情况...

In [1]:
import pandas as pd

In [2]:
malicious_urls = pd.read_csv('test.txt', sep='\t', header=None)

In [3]:
malicious_urls

,0,1
0,bd-un1.wanglv.com/auycbe.js,1
1,www.hzed.com/front/activity/verifyMobileForReg,1
2,47.93.177.253:4346/HZJH.aspx?Lottery=CQSSC&Mke...,1
3,115.231.155.32/ttyzcpzoomtmbcbohggmaztaspeannb...,1
4,222.73.132.173/zijian.hls.video.qq.com/DE26960...,1
...,...,...
2157,m.nbflq.com/js/error.js?20180608,1
2158,sinacloud.net/qunimg/rmsp.png,1
2159,61.147.235.246/dlied1.qq.com/iedsafe/sanlix/1a...,1
2160,www.qyffw.com/favicon.ico,1


再看个黑白样本的数量分布情况

In [4]:
malicious_urls[1].value_counts()

1    1794
0     368
Name: 1, dtype: int64

黑样本居多，但还不至于出现样本不均衡问题，所以在模型选择和评估方面，还不需要注意这方面的问题！

我们对URL排个序，看看该如何选择特征...

In [5]:
malicious_urls.sort_values(0)

,0,1
1921,1.1/android_update.htm?uid=9DC3091F90310C72FFE...,1
581,1.wenzhangba.cn/bwosfoscy.js,1
580,1.wenzhangba.cn/source/js/web/35zptm.js?p=kcgm...,1
752,112.117.221.11/qpdxv/v0/20160324/7b/7e/86b3cea...,1
2151,112.117.221.11/qpdxv/v0/20181108/d8/ea/348617c...,1
...,...,...
1099,zuikzy.com/?m=vod-detail-id-17332.html,1
886,zuqhxz.byrdyryr.com:43564/ccc.aspx?g=fb28gY+hQ...,1
320,zxcs.xiaodu9.cn/favicon.ico,1
1267,zxcs1.qlrtr.cn/orders/v2/pay?order_id=KHFZOK59...,1


大致看了下，该URLs数据集定义恶意URL的标准，并不同于直白的比如基于web攻击类型的定义，而是基本上是涉及诸如黄色图片，视频，赌球，涉政等问题，而对于这种问题的恶意定义都是主观性的，而如果使用特征抽取的传统机器学习方法来学习，特征的寻找相对来说比较广泛，所以想尝试使用深度学习中的类似情感分析的方法来解决这类问题，比如使用conv1d，或rnn，lstm等模型来处理，因这类文本在上下文之间并不需要存在明显的关联性，所以决定采用conv1d，文本无关联需求的1维卷积来处理这类文本，速度上也会快很多。

In [6]:
malicious_urls[0]

0                             bd-un1.wanglv.com/auycbe.js
1          www.hzed.com/front/activity/verifyMobileForReg
2       47.93.177.253:4346/HZJH.aspx?Lottery=CQSSC&Mke...
3       115.231.155.32/ttyzcpzoomtmbcbohggmaztaspeannb...
4       222.73.132.173/zijian.hls.video.qq.com/DE26960...
                              ...                        
2157                     m.nbflq.com/js/error.js?20180608
2158                        sinacloud.net/qunimg/rmsp.png
2159    61.147.235.246/dlied1.qq.com/iedsafe/sanlix/1a...
2160                            www.qyffw.com/favicon.ico
2161                                211.151.23.149/config
Name: 0, Length: 2162, dtype: object

In [7]:
from keras import layers, models, utils, preprocessing, callbacks
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe

Using TensorFlow backend.


In [8]:
max_words = 10000
tokenizer = preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(malicious_urls[0])
sequences = tokenizer.texts_to_sequences(malicious_urls[0])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

maxlen = 0
for i in sequences:
    if len(i) > maxlen:
        maxlen = len(i)
print('Max sequence length %s .' % maxlen)

Found 7758 unique tokens.
Max sequence length 141 .


In [9]:
def data():
    malicious_urls = pd.read_csv('test.txt', sep='\t', header=None)
    
    max_words = 10000
    tokenizer = preprocessing.text.Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(malicious_urls[0])
    sequences = tokenizer.texts_to_sequences(malicious_urls[0])
    word_index = tokenizer.word_index

    maxlen = 0
    for i in sequences:
        if len(i) > maxlen:
            maxlen = len(i)

    X = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen, padding='post')
    y = malicious_urls[1].values
    
    Xtrain, Xtest, ytrain, ytest = X[:2000], X[2000:], y[:2000], y[2000:]
    
    return Xtrain, ytrain, Xtest, ytest

In [10]:
callbacks_list = [
    callbacks.EarlyStopping(
    monitor='acc',
    patience=1,),
    callbacks.ModelCheckpoint(
    filepath='my_model1.h5',
    monitor='val_loss',
    save_best_only=True,)] 

def create_model(Xtrain, ytrain, Xtest, ytest):
    malicious_urls = pd.read_csv('test.txt', sep='\t', header=None)
    
    max_words = 10000
    tokenizer = preprocessing.text.Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(malicious_urls[0])
    sequences = tokenizer.texts_to_sequences(malicious_urls[0])
    word_index = tokenizer.word_index

    maxlen = 0
    for i in sequences:
        if len(i) > maxlen:
            maxlen = len(i)
            
    callbacks_list = [
        callbacks.EarlyStopping(
        monitor='acc',
        patience=1,),
        callbacks.ModelCheckpoint(
        filepath='my_model1.h5',
        monitor='val_loss',
        save_best_only=True,)] 
            
    model = models.Sequential()
    model.add(layers.Embedding(len(word_index) + 1, {{choice([64, 128])}}, input_length=maxlen))
    model.add(layers.Conv1D({{choice([64, 128])}}, {{choice([5, 7, 9])}}, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer={{choice(['rmsprop', 'adam', 'nadam'])}}, loss='binary_crossentropy', metrics=['accuracy'])
    result = model.fit(Xtrain, ytrain, epochs={{choice([10, 20, 50, 100])}}, batch_size={{choice([50, 100])}}, validation_split=0.1, callbacks=callbacks_list)
    
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [11]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='Malicious_URLs_Detection')

Xtrain, ytrain, Xtest, ytest = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(Xtest, ytest))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    from keras import layers, models, utils, preprocessing, callbacks
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Conv1D': hp.choice('Conv1D', [64, 128]),
        'Conv1D_1': hp.choice('Conv1D_1', [64, 128]),
        'Conv1D_2': hp.choice('Conv1D_2', [5, 7, 9]),
        'optimizer': hp.choice('optimizer', ['rmsprop', 'adam', 'nadam']),
        'epochs': hp.choice('epochs', [10, 20, 50, 10

Epoch 5/50                                                                   
 100/1800 [>.............................]                                   
 - ETA: 0s - loss: 0.0354 - acc: 1.0000                                      
                                                                             
1800/1800 [==============================]                                   
 - 0s 26us/step - loss: 0.0287 - acc: 0.9978 - val_loss: 0.0335 - val_acc: 0.9950

Epoch 6/50                                                                   
 100/1800 [>.............................]                                   
 - ETA: 0s - loss: 0.0122 - acc: 1.0000                                      
                                                                             
1800/1800 [==============================]                                   
 - 0s 25us/step - loss: 0.0146 - acc: 0.9983 - val_loss: 0.0191 - val_acc: 1.0000

Epoch 7/50                                            

 100/1800 [>.............................]                    
 - ETA: 0s - loss: 0.1804 - acc: 0.9600                       
                                                              
1800/1800 [==============================]                    
 - 0s 29us/step - loss: 0.1086 - acc: 0.9761 - val_loss: 0.0539 - val_acc: 0.9950

Epoch 5/20                                                    
 100/1800 [>.............................]                    
 - ETA: 0s - loss: 0.0426 - acc: 1.0000                       
                                                              
1800/1800 [==============================]                    
 - 0s 28us/step - loss: 0.0369 - acc: 0.9933 - val_loss: 0.0323 - val_acc: 0.9950

Epoch 6/20                                                    
 100/1800 [>.............................]                    
 - ETA: 0s - loss: 0.0146 - acc: 1.0000                       
                                                              
1800/1800 [====

                                                              
1800/1800 [==============================]                    
 - 0s 68us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0076 - val_acc: 1.0000

Epoch 8/100                                                   
  50/1800 [..............................]                    
 - ETA: 0s - loss: 6.4089e-04 - acc: 1.0000                   
                                                              
 950/1800 [==============>...............]                    
 - ETA: 0s - loss: 3.6878e-04 - acc: 1.0000                   
                                                              
1750/1800 [============================>.]                    
 - ETA: 0s - loss: 4.9429e-04 - acc: 1.0000                   
                                                              
1800/1800 [==============================]                    
 - 0s 65us/step - loss: 4.8501e-04 - acc: 1.0000 - val_loss: 0.0206 - val_acc: 0.9900

Best valida

In [12]:
callbacks_list = [
    callbacks.EarlyStopping(
    monitor='acc',
    patience=1,),
    callbacks.ModelCheckpoint(
    filepath='my_model2.h5',
    monitor='val_loss',
    save_best_only=True,)] 

model = models.Sequential()
model.add(layers.Embedding(len(word_index) + 1, 128, input_length=maxlen))
model.add(layers.SeparableConv1D(64, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=50, batch_size=100, validation_split=0.1, callbacks=callbacks_list)

Train on 1800 samples, validate on 200 samples
Epoch 1/50
1800/1800 [==============================] - 0s 167us/step - loss: 0.6240 - acc: 0.7956 - val_loss: 0.4551 - val_acc: 1.0000
Epoch 2/50
1800/1800 [==============================] - 0s 35us/step - loss: 0.5089 - acc: 0.7956 - val_loss: 0.2549 - val_acc: 1.0000


In [13]:
model.evaluate(Xtest, ytest)

162/162 [==============================] - 0s 33us/step


[0.25583459970391825, 1.0]

In [14]:
ypred = model.predict(Xtest)
for i, n in enumerate(ypred):
    if n > 0.5:
        ypred[i] = 1
    else:
        ypred[i] = 0

In [15]:
precision_score(ytest, ypred)

1.0

In [16]:
recall_score(ytest, ypred)

1.0

In [17]:
f1_score(ytest, ypred)

1.0

In [18]:
callbacks_list = [
    callbacks.EarlyStopping(
    monitor='acc',
    patience=1,),
    callbacks.ModelCheckpoint(
    filepath='my_model3.h5',
    monitor='val_loss',
    save_best_only=True,)]

model = models.Sequential()
model.add(layers.Embedding(len(word_index) + 1, 64, input_length=maxlen))
model.add(layers.Conv1D(64, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=10, batch_size=100, validation_split=0.1, callbacks=callbacks_list)

Train on 1800 samples, validate on 200 samples
Epoch 1/10
1800/1800 [==============================] - 0s 174us/step - loss: 0.5230 - acc: 0.7783 - val_loss: 0.1853 - val_acc: 1.0000
Epoch 2/10
1800/1800 [==============================] - 0s 24us/step - loss: 0.2624 - acc: 0.8783 - val_loss: 0.0746 - val_acc: 0.9950
Epoch 3/10
1800/1800 [==============================] - 0s 25us/step - loss: 0.0546 - acc: 0.9911 - val_loss: 0.0398 - val_acc: 0.9900
Epoch 4/10
1800/1800 [==============================] - 0s 24us/step - loss: 0.0160 - acc: 0.9972 - val_loss: 0.0225 - val_acc: 0.9950
Epoch 5/10
1800/1800 [==============================] - 0s 23us/step - loss: 0.0072 - acc: 0.9989 - val_loss: 0.0202 - val_acc: 0.9950
Epoch 6/10
1800/1800 [==============================] - 0s 24us/step - loss: 0.0039 - acc: 0.9994 - val_loss: 0.0221 - val_acc: 0.9950
Epoch 7/10
1800/1800 [==============================] - 0s 24us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0170 - val_acc: 0.9950
Epoch 8

In [19]:
model.evaluate(Xtest, ytest)

162/162 [==============================] - 0s 859us/step


[0.022874159508465247, 0.9876543209876543]

In [20]:
ypred = model.predict(Xtest)
for i, n in enumerate(ypred):
    if n > 0.5:
        ypred[i] = 1
    else:
        ypred[i] = 0

In [21]:
precision_score(ytest, ypred)

1.0

In [22]:
recall_score(ytest, ypred)

0.9876543209876543

In [23]:
f1_score(ytest, ypred)

0.9937888198757764

In [24]:
urls_lst = []
maxlen = 0
for url in malicious_urls[0].values:
    url_lst = []
    for char in url:
        try:
            url_lst.append(ord(char))
        except TypeError:
            url_lst.append(ord('?'))
    urls_lst.append(url_lst)
    getlen = len(url_lst)
    if getlen > maxlen:
        maxlen = getlen

In [25]:
X = preprocessing.sequence.pad_sequences(urls_lst, maxlen=maxlen, padding='post')
X = X.reshape((X.shape[0], X.shape[1], 1)) / 255.0
y = malicious_urls[1].values

In [26]:
Xtrain, Xtest, ytrain, ytest = X[:2000], X[2000:], y[:2000], y[2000:]

In [27]:
class ActivationLogger(callbacks.Callback):
    def set_model(self, model):
        self.model = model
        layer_output = model.layers[-1].output
        self.activations_model = models.Model(model.input, layer_output)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()

In [28]:
callbacks_list = [
    callbacks.EarlyStopping(
    monitor='acc',
    patience=1,),
    callbacks.ModelCheckpoint(
    filepath='my_model4.h5',
    monitor='val_loss',
    save_best_only=True,), ActivationLogger()] 

activation = ActivationLogger()

model = models.Sequential()
model.add(layers.Conv1D(64, 7, activation='relu', kernel_initializer='lecun_normal'))
model.add(layers.MaxPooling1D(7))
model.add(layers.Conv1D(64, 9, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=10, batch_size=100, validation_split=0.1, callbacks=[activation] + callbacks_list)

Train on 1800 samples, validate on 200 samples
Epoch 1/10
1800/1800 [==============================] - 0s 227us/step - loss: 0.4612 - acc: 0.7678 - val_loss: 0.1584 - val_acc: 0.9900
Epoch 2/10
1800/1800 [==============================] - 0s 45us/step - loss: 0.3996 - acc: 0.8272 - val_loss: 0.1453 - val_acc: 0.9700
Epoch 3/10
1800/1800 [==============================] - 0s 84us/step - loss: 0.3724 - acc: 0.8450 - val_loss: 0.1293 - val_acc: 0.9750
Epoch 4/10
1800/1800 [==============================] - 0s 44us/step - loss: 0.3532 - acc: 0.8583 - val_loss: 0.3918 - val_acc: 0.8400
Epoch 5/10
1800/1800 [==============================] - 0s 52us/step - loss: 0.3414 - acc: 0.8639 - val_loss: 0.0974 - val_acc: 0.9750
Epoch 6/10
1800/1800 [==============================] - 0s 105us/step - loss: 0.3294 - acc: 0.8639 - val_loss: 0.2553 - val_acc: 0.9350
Epoch 7/10
1800/1800 [==============================] - 0s 107us/step - loss: 0.2994 - acc: 0.8867 - val_loss: 0.1615 - val_acc: 0.9700
Epoch

In [29]:
model.evaluate(Xtest, ytest)

162/162 [==============================] - 0s 904us/step


[0.17353871943038188, 0.9567901234567902]

In [30]:
ypred = model.predict(Xtest)
for i, n in enumerate(ypred):
    if n > 0.5:
        ypred[i] = 1
    else:
        ypred[i] = 0

In [31]:
precision_score(ytest, ypred)

1.0

In [32]:
recall_score(ytest, ypred)

0.9567901234567902

In [33]:
f1_score(ytest, ypred)

0.9779179810725552

以上模型使用了两种方法，但都是基于一维卷积进行的：

1. 基于一维卷积的文本序列，将所有文本都转换为整数型数据，然后再结合词嵌入进行词向量的降维得到；
2. 思想是将每个文本序列转换为ascii码，因ascii码的范围恰遇图像的RGB和灰度范围都一致，将其看做是一个深度为1的灰度图，再使用一维卷积

整体来看，使用方法的效果模型更好，速度快，模型更简单，得到的精度也很高，而方法二的思路相对新颖，但模型相比方法一较复杂，训练时间也较长，，对于两个模型的解释，主要还是在一维卷积这种对于无序小数据文本处理的优势上！

In [34]:
utils.plot_model(model, show_shapes=True, to_file='model.png')